In [1]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import Chroma
from langchain_community import embeddings
from langchain_community.embeddings import GPT4AllEmbeddings
from langchain_community.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import PromptTemplate
from langchain_community.document_loaders import PyPDFLoader

from langchain.output_parsers import PydanticOutputParser
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import BSHTMLLoader

In [2]:
from langchain_community.document_loaders import UnstructuredHTMLLoader

In [3]:
loader = BSHTMLLoader("C:\\Users\\Barani\\Desktop\\local_ollama\\llm_testing_urls\\test_7.html")

In [4]:
data = loader.load()

In [5]:
data

[Document(page_content="\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n   Contact Ã…bo Akademi University | Ã…bo Akademi University\n  \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n        This website uses cookies\n       \n\n\n\n        We use cookies to personalise content, to provide social media features and to   analyse our traffic. Third parties whose services we use (like Google,   YouTube and Giosg) may gather information about your use of our site and may   combine it with other information that youâ€™ve provided to them or that   theyâ€™ve collected from your use of their services.\n       \n\n\n\n\n\n\n\n          Allow all\n         \n\n\n\n          Allow only necessary cookies\n         \n\n\n\n          Show details\n         \n\n\n\n\n\n\n\n\n\n\n         Allow my selection of cookies\n        \n\n\n\n\n\n\n\n\n\n\n             Necessary (23)\n            \n\n\n\n           Show details\n           \n

In [6]:
model_local = ChatOllama(model="mistral",temperature=0)

In [14]:
# loader = PyPDFLoader("C:\\Users\\Barani\\Desktop\\local_ollama\\pubmed_cleaning_research_paper.pdf")
# data = loader.load_and_split()

In [15]:
# print(data)

In [7]:
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(chunk_size=7500, chunk_overlap=100)
doc_splits = text_splitter.split_documents(data)

In [8]:
embeddings = GPT4AllEmbeddings()

# Create the Chroma vector store
vectorstore = Chroma.from_documents(
    documents=doc_splits,
    collection_name="rag-chroma",
    embedding=embeddings
)

# Create the retriever
retriever = vectorstore.as_retriever()

In [18]:
# 4. After RAG
print("\n########\nAfter RAG\n")
after_rag_template = """Answer the question based only on the following context:
{context}
Question: {question}
"""
after_rag_prompt = ChatPromptTemplate.from_template(after_rag_template)
after_rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | after_rag_prompt
    | model_local
    | StrOutputParser()
)
print(after_rag_chain.invoke("provide address,email address,contact number from the given context"))

Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3



########
After RAG

 I apologize, but there is no specific address, email address, or contact number mentioned in the provided context. The context only includes information about cookies used by YouTube on a webpage.


In [ ]:
after_rag_chain.invoke("explain the affiliation normalization process code which is given in the context")